In [1]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
import gensim
from gensim.models import CoherenceModel
from gensim import corpora
import pandas as pd
from pprint import pprint
import string
import os
import re
import  sys
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis


In [ ]:
!pip install pyLDAvis

In [2]:
# Add parent directory to path to import modules from src
rpath = os.path.abspath('C:/Users/moyka/OneDrive/Documents/GitHub/week0_starter_network_analysis')
if rpath not in sys.path:
    sys.path.insert(0, rpath)

from src.loader import SlackDataLoader
import src.utils as utils

In [3]:
data_loader = SlackDataLoader(r"C:/Users/moyka/Desktop/Dashboard/anonymized/")

def get_channel_messages(channel):
    channel_messages = utils.get_messages_on_channel(f"C:/Users/moyka/Desktop/Dashboard/anonymized/{channel}") 
    # Create an empty DataFrame
    df = pd.DataFrame(channel_messages)
    return df

def get_all_channels_message():
    dfs = []  # List to store individual DataFrames

    for channel in data_loader.channels:
        dfs.append(get_channel_messages(channel["name"]))

    # Concatenate all DataFrames into a single DataFrame
    result_df = pd.concat(dfs, ignore_index=True)

    return result_df

In [4]:
def preprocess_text(text):
    # Extract and remove URLs
    urls = re.findall(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', text)
    for url in urls:
        text = text.replace(url, '')

    text = re.sub(r'<@.*?>', '', text)

    # Convert to lowercase
    text = text.lower()

    # Remove punctuation
    text = ''.join([char for char in text if char not in string.punctuation])

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Tokenize
    tokens = word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]

    # Perform stemming
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]

    # Perform lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    # Join the tokens back into a string
    text = ' '.join(tokens)

    return text


In [5]:

def prepare_data(df):
    df['cleaned_text'] = df['text'].apply(preprocess_text)
    sentence_list = [tweet for tweet in df['cleaned_text']]
    word_list = [sent.split() for sent in sentence_list]

    #Create dictionary which contains Id and word
    word_to_id = corpora.Dictionary(word_list) #generate unique tokens
    corpus = [word_to_id.doc2bow(tweet) for tweet in word_list]
    
    return df, word_list, word_to_id, corpus

In [6]:
def build_model(corpus, word_to_id):
    # Build LDA model
    lda_model = gensim.models.ldamodel.LdaModel(corpus,
                                        id2word=word_to_id,
                                        num_topics=5,
                                        random_state=100,
                                        update_every=1,
                                        chunksize=100,
                                        passes=10,
                                        alpha='auto',
                                        per_word_topics=True)    
    return lda_model

In [7]:

def show_topics(lda_model):
    pprint(lda_model.show_topics(formatted=False))

In [8]:
def model_analysis(lda_model, corpus, word_list, word_to_id):
    print('\nPerplexity: ', lda_model.log_perplexity(corpus))
    doc_lda = lda_model[corpus]


    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=word_list, dictionary=word_to_id, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    print('\n Lda model Coherence Score/Accuracy on Tweets: ', coherence_lda)

In [9]:
def get_top_topics(df):
    df, word_list, word_to_id, corpus = prepare_data(df)
    lda_model = build_model(corpus, word_to_id)

    # Show the top 10 topics
    show_topics(lda_model)
    

    # Visualize the top 10 topics
    pyLDAvis.enable_notebook()
    LDAvis_prepared = gensimvis.prepare(lda_model, corpus, word_to_id)
    return LDAvis_prepared

### Top 10 topics of the different channels

In [11]:
df = get_channel_messages("all-week1")
get_top_topics(df)

AttributeError: module 'src.utils' has no attribute 'get_messages_on_channel'

In [12]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\moyka\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [13]:
df = get_channel_messages("all-community-building")
get_top_topics(df)

AttributeError: module 'src.utils' has no attribute 'get_messages_on_channel'

### Top 10 topics of all channels

In [14]:
df = get_all_channels_message()
get_top_topics(df)

AttributeError: module 'src.utils' has no attribute 'get_messages_on_channel'